# Denison DA210/CS181 SW Lab #12 - Step 2

Before you get your checkpoints, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells**.

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules

In [ ]:
import pandas as pd
import os
import os.path
import json
import sqlalchemy as sa

#### Set credentials

In [ ]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [ ]:
def buildConnectionString(source="sqlite_book"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

cstring = buildConnectionString("sqlite_book")
print("Connection string:", cstring)

---

## Part D: Using Variables with template strings

We already know how to use Python string formatting to build template strings.  We can use this strategy to write SQL queries that use variables for the source of values (e.g., in the `WHERE` clause):

In [ ]:
# Re-connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

In [ ]:
# Build a query template string
query_template = """
SELECT code, pop, gdp, life
FROM indicators
WHERE year = {} AND life > {}
"""

In [ ]:
# Use the template with year=2010 and threshold=82
query_2010 = query_template.format(2010, 82)
df1 = pd.read_sql_query(query_2010, con=connection)
df1

In [ ]:
# Use the template with year=2017 and threshold=84
query_2017 = query_template.format(2017, 84)
df2 = pd.read_sql_query(query_2017, con=connection)
df2

We could, in fact, write a function to abstract performing this query.  In the SQL query below, we also join with `countries` to get the name of the countries that pass the filters.

In [ ]:
def indicators_life(dbcon, year=2017, threshold=0):
    query_template = """
    SELECT i.code, country, pop, gdp, life
    FROM indicators AS i
        INNER JOIN countries USING (code)
    WHERE year = {} AND life > {}
    """

    query = query_template.format(year, threshold)
    return pd.read_sql_query(query, con=connection)

In [ ]:
df3 = indicators_life(connection, year=2000, threshold=80)
df3

In [ ]:
df4 = indicators_life(connection, year=1980, threshold=76)
df4

In [ ]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

---

#### Try it yourself!

Now, let's get some practice using the `school` database.

In [ ]:
# Build the connection string
cstring = buildConnectionString("sqlite_school")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

**Q3:** Write a function
```
    departmentsByDivision(connection, division)
```
that queries the `departments` table for the set of departments (all fields) in the given `division`.  Use Python string composition and the `format()` method (or Python `f strings`) for building the query.  Return a `pandas` `DataFrame`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Debugging cell
df_humanities = departmentsByDivision(connection, "Humanities")
df_humanities

In [ ]:
# Testing cell
df_finearts = departmentsByDivision(connection, "Fine Arts")
assert df_finearts.shape == (5, 4)
assert "Cinema" in list(df_finearts["departmentname"])

df_inter = departmentsByDivision(connection, "Interdisciplinary")
assert df_inter.shape == (13, 4)
assert "Biology" not in list(df_inter["departmentname"])

In [ ]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine